In [13]:
import pandas as pd
import data_util
import numpy as np

import torch
from transformers import BertTokenizer
from keras_preprocessing.sequence import pad_sequences
# from transformers import BertForSequnceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [2]:
train_data, test_data = data_util.load_data()

reading data


In [4]:
train_data

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
...,...,...,...
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1


In [5]:
train_data, val_data = np.split(train_data.sample(frac=1), [int(0.8*len(train_data))])

In [10]:
def tokenize(tokenizer, sentences, MAX_LEN):

    input_ids = []

    for i, sent in enumerate(sentences):
        if (i + 1) % 1000 == 0:
            print("tokenizing " + str(i + 1) + " out of " + str(len(sentences)))
        encoded_sent = tokenizer.encode(
                sent,
                add_special_tokens=True,
                max_length = 128, # 32 / 64 / 128 / none
                pad_to_max_length = True
                )
        input_ids.append(encoded_sent)

    #  pdb.set_trace()

    MAX_LEN = max([len(sen) for sen in input_ids])

    print("padding...")
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
    
    attention_masks = []
    for sent in input_ids:
        att_mask = [int(token_id > 0) for token_id in sent]
        attention_masks.append(att_mask)

    return input_ids, MAX_LEN, attention_masks

In [14]:
train_data[:5]

,id,document,label
1349,4546653,이거야 이거 ㅋㅋㅋㅋㅋ,1
66536,89778,눈물이 끊이지 않는 아름다운 영화다,1
84466,10191313,재미 없음 감동 없음 내용 없음 개연성 없음 시간 존나게 아까움 돈도 아까움 별점도...,0
54310,9726795,프레데터 나오는 부분은 옛날 파워레인저나 백터맨의 크리쳐 나오는 줄 이거 에일리언보...,0
52936,2708404,액션영화답지않은 지루한전개에 시걸의 얼토당토않은 명언은 더욱 최악 ㅡㅡ,0


In [16]:
MAX_LEN = 64
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

train_inputs, train_MAX_LEN, train_attn_masks = tokenize(tokenizer, train_data['document'],MAX_LEN)
train_labels = train_data['label'].values

tokenizing 1000 out of 116632
tokenizing 2000 out of 116632
tokenizing 3000 out of 116632
tokenizing 4000 out of 116632
tokenizing 5000 out of 116632
tokenizing 6000 out of 116632
tokenizing 7000 out of 116632
tokenizing 8000 out of 116632
tokenizing 9000 out of 116632
tokenizing 10000 out of 116632
tokenizing 11000 out of 116632
tokenizing 12000 out of 116632
tokenizing 13000 out of 116632
tokenizing 14000 out of 116632
tokenizing 15000 out of 116632
tokenizing 16000 out of 116632
tokenizing 17000 out of 116632
tokenizing 18000 out of 116632
tokenizing 19000 out of 116632
tokenizing 20000 out of 116632
tokenizing 21000 out of 116632
tokenizing 22000 out of 116632
tokenizing 23000 out of 116632
tokenizing 24000 out of 116632
tokenizing 25000 out of 116632
tokenizing 26000 out of 116632
tokenizing 27000 out of 116632
tokenizing 28000 out of 116632
tokenizing 29000 out of 116632
tokenizing 30000 out of 116632
tokenizing 31000 out of 116632
tokenizing 32000 out of 116632
tokenizing 33000 

In [17]:
train_inputs[:5]

array([[  101,   100,  1463, 30019, 29991, 30008,   100,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [18]:
train_labels[:5]

array([1, 1, 0, 0, 0])